In [121]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 

In [122]:
df_1.sample(10)

,country,age_group,sex,marital_status,data_process,data_collection_start_year,data_collection_end_year,data_source
102842,Honduras,[60-64],Man,Consensual union,Census,1988,1988,US Census Bureau
191850,Peru,[20-24],Woman,Married,Survey,2000,2000,DHS_STATcompiler
2231,American Samoa,[15-19],Woman,Single,Census,1974,1974,UNSD
34677,Canada,[25-29],Woman,Single,Estimate,1998,1998,UNSD
194371,Philippines,[25-29],Woman,Single,Census,2000,2000,UNSD
168908,Nepal,[20-24],Man,Not living together,Survey,2016,2017,DHS_STATcompiler
118487,Indonesia,[60-64],Man,Married,Survey,2017,2017,DHS_HH
199328,Republic of Korea,[30-34],Woman,Widowed,Census,1970,1970,UNSD
130349,Japan,[45-49],Woman,Single,Census,1990,1990,UNSD
110476,Iceland,[40-44],Man,Single,Estimate,1981,1981,UNSD


In [123]:
df_1.sample(5)

,country,age_group,sex,marital_status,data_process,data_collection_start_year,data_collection_end_year,data_source
62383,Denmark,[35-39],Woman,Divorced,Estimate,1997,1997,UNSD
33008,Canada,[45-49],Woman,Single,Estimate,1986,1986,UNSD
214914,Saudi Arabia,[25-29],Woman,Widowed,Estimate,2017,2017,UNSD
84209,France,[10-14],Man,Single,Estimate,2001,2001,UNSD
188667,Palau,[15-19],Man,Separated,Census,1995,1995,UNSD


In [128]:
print(df_1.dtypes)

country                       object
age_group                     object
sex                           object
marital_status                object
data_process                  object
data_collection_start_year     int32
data_collection_end_year       int32
data_source                   object
dtype: object


In [129]:
df_2.sample(5)

,country,code,year,mean_age_of_women_at_first_marriage
448,Israel,ISR,2003,24.3
328,Germany,DEU,1995,26.7
304,France,FRA,1999,28.2
390,Hungary,HUN,2001,25.2
847,Slovenia,SVN,2017,30.4


In [131]:
df_2.dtypes

country                                 object
code                                    object
year                                     int32
mean_age_of_women_at_first_marriage    float64
dtype: object

In [132]:
df_3.sample(5)

,country,code,year,crude_marriage_rate_marriages_per_1000_people
1113,Italy,ITA,1970,7.3
1150,Italy,ITA,2007,4.3
2065,Spain,ESP,1981,5.4
1059,Ireland,IRL,2013,4.5
864,Greece,GRC,2001,5.3


In [135]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
640,Denmark,DNK,1984,5.6,6.1,1990
1175,Ireland,IRL,2014,4.7,5.1,1990
45,Australia,AUS,2003,5.3,6.9,1990
865,Germany,DEU,1973,6.7,6.5,1990
1187,Israel,ISR,1987,6.9,6.8,1990
1526,Luxembourg,LUX,1998,4.8,6.1,1990


In [137]:
df_4.sample(6)

,country,code,year,crude_marriage_rate,crude_marriage_rate_people1,year_1
387,Costa Rica,CRI,1980,7.6,7.5,1990
1639,Mexico,MEX,2003,5.6,7.4,1990
417,Costa Rica,CRI,2010,5.3,7.5,1990
706,Estonia,EST,1976,8.6,7.5,1990
40,Australia,AUS,1998,5.9,6.9,1990
2515,United Kingdom,GBR,1978,7.4,6.6,1990


In [138]:
df_5.sample(5)

,country,code,year,share_of_births_outside_of_marriage
2066,United Kingdom,GBR,2010,46.9
829,Hungary,HUN,1986,9.5
1171,Lithuania,LTU,1991,7.0
637,Finland,FIN,2000,39.2
1930,Sweden,SWE,2006,55.5
